### Import the necessary modules

In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

### Get the first page of computer products

In [2]:
amazon_url = 'https://www.amazon.com.tr/'
driver = webdriver.Edge(executable_path='drivers/msedgedriver.exe')

driver.get(amazon_url)
button = driver.find_element_by_css_selector('a[data-csa-c-content-id=nav_cs_pc]')

# The title 'Dizüstü Bilgisayarlar' would be different for the languages other than turkish

driver.get(button.get_attribute('href'))
button = driver.find_element_by_css_selector('a[title="Dizüstü Bilgisayarlar"]')

driver.get(button.get_attribute('href'))
button = driver.find_element_by_css_selector('.apb-browse-searchresults-footer > div > a')

driver.get(button.get_attribute('href'))
page_limit = int(driver.find_element_by_css_selector('li[aria-disabled=true]').get_attribute('innerHTML'))

### Collect all links to the products from pages

In [3]:
links = [e.get_attribute('href') for e in driver.find_elements_by_css_selector('span[data-component-type=s-product-image] > a')]

for i in range(1, page_limit):
    next_button = driver.find_element_by_css_selector('.a-last > a')
    driver.get(next_button.get_attribute('href'))
    links += [e.get_attribute('href') for e in driver.find_elements_by_css_selector('span[data-component-type=s-product-image] > a')]

### Visit the links and get the product details

In [4]:
product_list = []

for link in links:
    driver.get(link)
    
    try:
        price = driver.find_element_by_id('price_inside_buybox')
        product_table = driver.find_element_by_id('productDetails_techSpec_section_1')
    except NoSuchElementException:
        continue
        
    try:
        score = driver.find_element_by_css_selector('span[data-hook=rating-out-of-text]')
    except NoSuchElementException:
        score = None
    
    product_data = {row.find_element_by_tag_name('th').get_attribute('innerHTML').strip(): row.find_element_by_tag_name('td').get_attribute('innerHTML').strip() for row in product_table.find_elements_by_tag_name('tr')}
    product_data['Price'] = price.get_attribute('innerHTML').strip()
    product_data['Score'] = score.get_attribute('innerHTML') if score else None
    product_list.append(product_data)

### Create a dataframe from the data

In [5]:
df_computer = pd.DataFrame()
for product in product_list:
    df_computer = df_computer.append(pd.Series(product), ignore_index=True)
    
df_computer.to_csv('computer.csv', index=False)